# Using a Temporal Convolutional Network for Daytrading
### Daniel Kalam, Sharvita Paithankar

In [53]:
import tensorflow as tf
import pandas as pd
from pandas_datareader import DataReader
import numpy as np
import matplotlib.pyplot as plt
import datetime

### Gathering Data
Getting data for 100 stocks in the date range of April 2nd, 2018 to October 9th, 2020 from yahoo finance.

In [69]:
symbols = ['AAPL', 'TSLA', 'FB', 'GE', 'BRK', 'GOOGL', 'INTC', 'AMD', 'HPE', 'ZM',
          'CAKE', 'AET', 'F', 'KO', 'DDS', 'NVDA', 'NFLX', 'JPM', 'AMZN', 'MSFT']
#TODO: Add 80 more symbols.
source = 'yahoo'
start_date = pd.to_datetime('2018-02-04')
end_date = pd.to_datetime('2020-10-09')
stock_data = {}
for symbol in symbols:
    stock_data[symbol] = DataReader(symbol, source, start_date, end_date)

Create a data frame for each column in a stock's data frame.

In [72]:
open_data = {}
close_data = {}
high_data = {}
low_data = {}
volume_data = {}
adj_close_data = {}
for symbol in stock_data:
    open_data[symbol] = stock_data[symbol].Open
    close_data[symbol] = stock_data[symbol].Close
    high_data[symbol] = stock_data[symbol].High
    low_data[symbol] = stock_data[symbol].Low
    volume_data[symbol] = stock_data[symbol].Volume
    adj_close_data[symbol] = stock_data[symbol]['Adj Close']

### TensorFlow
Turn the data frames into tensorflow datatypes so that they can be processed by tensorflow.

In [ ]:
class DilatedCausalConvolutionalNetwork(tf.keras.layers.Conv1D):
    